In [1]:
%load_ext autoreload
%autoreload 2

from chartbot_config import *

In [6]:
import pandas as pd

df = pd.read_csv(raw_data_path, encoding = 'unicode_escape')
df = df.melt(id_vars=["Domain", "Sub domain", "Intent", "Answer Format"]).drop("variable", axis = 1)

Classes_dict = dict(zip(list(df["Intent"].unique()), [i for i in range(df["Intent"].nunique())]))
df["labels"] = df.Intent.apply(lambda x: Classes_dict.get(x))
df["labels"] = pd.get_dummies(df.labels).values.tolist()
df = df.dropna().reset_index().copy()
df.head()

,index,Domain,Sub domain,Intent,Answer Format,value,labels
0,0,About Techology,AI Techology,AI_summary,We utilise AI technology in serveal ways to im...,Why Did You Use AI In Your Service?,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,About Techology,AI Techology,AI_component,"Currently, we introduced the ""Form Recogniser""...",What is you doing with AI technology?,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,About Techology,iAM SMART Integration,Without_iamsmart,You can still use our service without using iA...,I do not want to register an iAMSTART account....,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,About Techology,Data Security & Privacy,data_removal,"Go to XXX pages, click on XXX button. Please n...",Where can I find a Delete button for my data?,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,About Business Component,Account,Delete_account,"Go to the XXX pages, click XXX button. Your ac...",Can You Removing My Account?,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
from datasets import Dataset

df = Dataset.from_pandas(df)
df = df.train_test_split(train_size = 1 - test_size , seed=seed)
df_splitted = df["train"].train_test_split(train_size= (1 - validation_size - test_size)/(1 - test_size)  , seed=seed)
df_splitted["validation"] = df_splitted.pop("test")
df_splitted["test"] = df["test"]
df_splitted

D:\Anaconda3\envs\GPU\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['index', 'Domain', 'Sub domain', 'Intent', 'Answer Format', 'value', 'labels'],
        num_rows: 9766
    })
    validation: Dataset({
        features: ['index', 'Domain', 'Sub domain', 'Intent', 'Answer Format', 'value', 'labels'],
        num_rows: 3256
    })
    test: Dataset({
        features: ['index', 'Domain', 'Sub domain', 'Intent', 'Answer Format', 'value', 'labels'],
        num_rows: 3256
    })
})

In [10]:
df_splitted.save_to_disk(processed_data_path)

Flattening the indices: 100%|██████████| 4/4 [00:00<00:00, 38.83ba/s]
